In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-qj01oen9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-qj01oen9
  Resolved https://github.com/unslothai/unsloth.git to commit 05f4875aff111bf3801f6e740cc03cb7a8594c9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192302 sha256=7b5b51b4b2bc441f668d9d6b708e90b9ecafc2a38d9cb9734c57731e28d3bdc6
  Stored in directory: /tmp/pip-ephem-wheel-cache-mdadhwft/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.3.19
    Uninstalling unsloth-2025.3.19:
      Successfully uninstalled unsloth-2025.3.19


In [ ]:
from huggingface_hub import login
login()
import wandb
wandb.login()

In [ ]:
# === 步骤 1：加载模型和分词器 ===
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # 基座模型，1.5B 参数，适合微调
    max_seq_length=2048,  # 最长序列长度，支持长上下文输入
    load_in_4bit=True,  # 4 位量化，降低内存占用（约 2-3 GB），适合 Colab T4
    dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,  # 使用 bfloat16（若支持）以提高精度
    trust_remote_code=True,  # 允许加载 DeepSeek 的自定义代码
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("jnmil/safety", split="train")

safety_helmet_qa.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/497 [00:00<?, ? examples/s]

In [ ]:
# 定义提示模板，使用中文注释以清晰表达意图
train_prompt_style = """以下是一个任务描述和输入上下文的指令。请撰写一个合适的回答。在回答之前，仔细思考并构建逐步推理链，确保回答逻辑清晰、准确。

### Instruction：
您是建筑安全领域的专家，特别是在安全帽使用方面。请回答以下问题。

### Question：
{}
### Response：
<think>
{}
</think>
{}"""

In [ ]:
# 格式化数据集函数，将问题、推理链和回答组合为完整文本
def formatting_prompts_func(examples):
    inputs = examples["Question"]  # 问题字段
    cots = examples["Complex_CoT"]  # 推理链字段
    outputs = examples["Response"]  # 回答字段
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 按模板格式化，添加 EOS 标记以结束序列
        text = train_prompt_style.format(input, cot, output) + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

In [ ]:
# 应用格式化，批处理提高效率
dataset = dataset.map(formatting_prompts_func, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/497 [00:00<?, ? examples/s]

In [ ]:
# === 步骤 3：配置 LoRA 微调 ===
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA 秩，增加到 32 增强适配能力，权衡内存（约增加 0.5 GB）
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # 注意力模块
        "gate_proj", "up_proj", "down_proj"  # MLP 模块
    ],  # 针对 Transformer 核心层，全面优化
    lora_alpha=32,  # 缩放因子，与 r 匹配，增强 LoRA 效果
    lora_dropout=0,  # 无 dropout，微调无需正则化
    bias="none",  # 无偏置，减少参数量
    use_gradient_checkpointing="unsloth",  # Unsloth 优化检查点，降低 30% 内存
    random_state=3407,  # 固定随机种子，确保可重复性
    use_rslora=True,  # 使用 Rank-Stabilized LoRA，稳定训练
    loftq_config=None,  # 不使用 LoFTQ，保持标准 LoRA
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=1e-4,
        max_steps=1000,
        warmup_steps=10,
        lr_scheduler_type="cosine",
        optim="adamw_8bit",
        weight_decay=0.01,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        save_steps=200,
        output_dir="outputs",
        save_total_limit=2,
        eval_strategy="no",  # 禁用验证
        seed=3407,
        report_to="none",
    ),
)

In [ ]:
# === 步骤 5：开始训练 ===
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 497 | Num Epochs = 17 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 36,929,536/5,000,000,000 (0.74% trained)


Step,Training Loss
10,3.327300
20,1.056400
30,0.763500
40,0.694200
50,0.630700
60,0.662800
70,0.544300
80,0.394600
90,0.393300
100,0.431700


TrainOutput(global_step=1000, training_loss=0.17038934117555618, metrics={'train_runtime': 926.6555, 'train_samples_per_second': 8.633, 'train_steps_per_second': 1.079, 'total_flos': 8546060727152640.0, 'train_loss': 0.17038934117555618})

In [ ]:
new_model_local = "DeepSeek-R1-Construction-Safety-f16"
model.save_pretrained("construction-safety-lora")  # 保存 LoRA 适配器
tokenizer.save_pretrained("construction-safety-lora")  # 保存分词器

('construction-safety-lora/tokenizer_config.json',
 'construction-safety-lora/special_tokens_map.json',
 'construction-safety-lora/tokenizer.json')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="jnmil/DeepSeek-R1-Construction-Safety-f16")
model.push_to_hub("jnmil/DeepSeek-R1-Construction-Safety-f16")
tokenizer.push_to_hub("jnmil/DeepSeek-R1-Construction-Safety-f16")

adapter_model.safetensors:   0%|          | 0.00/148M [00:00<?, ?B/s]

Saved model to https://huggingface.co/jnmil/DeepSeek-R1-Construction-Safety-f16


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="jnmil/DeepSeek-R1-Construction-Safety-f16", local_dir="/content/DeepSeek-R1-Construction-Safety-f16")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/148M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

'/content/DeepSeek-R1-Construction-Safety-f16'

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 48674, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 48674 (delta 32), reused 25 (delta 25), pack-reused 48628 (from 3)
Receiving objects: 100% (48674/48674), 102.59 MiB | 24.14 MiB/s, done.
Resolving deltas: 100% (35083/35083), done.


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
%cd /content/llama.cpp

/content/llama.cpp


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu


In [ ]:
!ls -l /content/DeepSeek-R1-Construction-Safety-f16

total 155488
-rw-r--r-- 1 root root       824 Apr 14 02:04 adapter_config.json
-rw-r--r-- 1 root root 147770496 Apr 14 02:04 adapter_model.safetensors
-rw-r--r-- 1 root root      5181 Apr 14 02:04 README.md
-rw-r--r-- 1 root root       472 Apr 14 02:04 special_tokens_map.json
-rw-r--r-- 1 root root      6783 Apr 14 02:04 tokenizer_config.json
-rw-r--r-- 1 root root  11422778 Apr 14 02:04 tokenizer.json


In [ ]:
!pip install transformers peft torch huggingface_hub

In [ ]:
!pip install --upgrade huggingface_hub transformers peft

In [ ]:
!pip install transformers peft torch huggingface_hub

In [ ]:
!pip show transformers
!pip show peft
!pip show torch

Name: transformers
Version: 4.51.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
Name: peft
Version: 0.15.1
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: benjamin@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: accelerate, huggingface_hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 
Name: torch
Version: 2.2.2+cpu


In [ ]:
!pip uninstall -y transformers peft torch torchvision torchaudio

Found existing installation: transformers 4.51.2
Uninstalling transformers-4.51.2:
  Successfully uninstalled transformers-4.51.2
Found existing installation: peft 0.15.1
Uninstalling peft-0.15.1:
  Successfully uninstalled peft-0.15.1
Found existing installation: torch 2.2.2+cpu
Uninstalling torch-2.2.2+cpu:
  Successfully uninstalled torch-2.2.2+cpu
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [ ]:
!pip install transformers==4.44.2 peft==0.12.0 torch==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 788.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!python /content/llama.cpp/convert_hf_to_gguf.py /content/DeepSeek-R1-Construction-Safety-merged-f16 --outfile /content/DeepSeek-R1-Construction-Safety-f16.gguf --outtype f16

In [ ]:
!cat /content/DeepSeek-R1-Construction-Safety-f16/adapter_config.json

{
  "alpha_pattern": {},
  "auto_mapping": null,
  "base_model_name_or_path": "unsloth/deepseek-r1-distill-qwen-1.5b-unsloth-bnb-4bit",
  "bias": "none",
  "eva_config": null,
  "exclude_modules": null,
  "fan_in_fan_out": false,
  "inference_mode": true,
  "init_lora_weights": true,
  "layer_replication": null,
  "layers_pattern": null,
  "layers_to_transform": null,
  "loftq_config": {},
  "lora_alpha": 32,
  "lora_bias": false,
  "lora_dropout": 0,
  "megatron_config": null,
  "megatron_core": "megatron.core",
  "modules_to_save": null,
  "peft_type": "LORA",
  "r": 32,
  "rank_pattern": {},
  "revision": null,
  "target_modules": [
    "gate_proj",
    "q_proj",
    "up_proj",
    "down_proj",
    "o_proj",
    "k_proj",
    "v_proj"
  ],
  "task_type": "CAUSAL_LM",
  "use_dora": false,
  "use_rslora": true
}

In [ ]:
import json

# 读取配置文件
config_path = "/content/DeepSeek-R1-Construction-Safety-f16/adapter_config.json"
with open(config_path, "r") as f:
    config = json.load(f)

# 删除 eva_config
if "eva_config" in config:
    del config["eva_config"]
    print("已删除 eva_config 字段")
else:
    print("未找到 eva_config 字段")

# 保存修改后的配置文件
with open(config_path, "w") as f:
    json.dump(config, f, indent=2)

print("配置文件已更新")

已删除 eva_config 字段
配置文件已更新


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 设置基座模型名称
base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
lora_model_path = "/content/DeepSeek-R1-Construction-Safety-f16"
merged_model_path = "/content/DeepSeek-R1-Construction-Safety-merged-f16"

# 加载基座模型
print("加载基座模型...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True
)

# 加载 LoRA 适配器
print("加载 LoRA 适配器...")
model = PeftModel.from_pretrained(model, lora_model_path)

# 合并模型
print("合并模型...")
merged_model = model.merge_and_unload()

# 保存合并模型
print("保存合并模型...")
merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

print(f"合并模型已保存到 {merged_model_path}")

加载基座模型...
加载 LoRA 适配器...
合并模型...
保存合并模型...
合并模型已保存到 /content/DeepSeek-R1-Construction-Safety-merged-f16


In [ ]:
!python /content/llama.cpp/convert_hf_to_gguf.py /content/DeepSeek-R1-Construction-Safety-merged-f16 --outfile /content/DeepSeek-R1-Construction-Safety-f16.gguf --outtype f16

INFO:hf-to-gguf:Loading model: DeepSeek-R1-Construction-Safety-merged-f16
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,             torch.float16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float16 --> F16, shape = {8960, 1536}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.float16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,     torch.float16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.0.attn_k.bias,         torch.float16 --> F32, shape = {256}
INFO:hf-to-gguf:blk.0.attn_k.weight,       torch.float16 --> F16, shape = {1536,

In [ ]:
!ls -lh /content/DeepSeek-R1-Construction-Safety-f16.gguf

-rw-r--r-- 1 root root 3.4G Apr 14 02:08 /content/DeepSeek-R1-Construction-Safety-f16.gguf


In [ ]:
from huggingface_hub import HfApi, login
login()  # 输入 token
api = HfApi()
api.create_repo(repo_id="jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF", repo_type="model")

RepoUrl('https://huggingface.co/jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF')

In [ ]:
api.upload_file(
    path_or_fileobj="/content//DeepSeek-R1-Construction-Safety-f16.gguf",
    path_in_repo="DeepSeek-R1-Construction-Safety-f16.gguf",
    repo_id="jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF",
    repo_type="model"
)

DeepSeek-R1-Construction-Safety-f16.gguf:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF/commit/2436b103999ecc06893acb00b0be51475b7bbcd1', commit_message='Upload DeepSeek-R1-Construction-Safety-f16.gguf with huggingface_hub', commit_description='', oid='2436b103999ecc06893acb00b0be51475b7bbcd1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='jnmil/DeepSeek-R1-Construction-Safety-f16-GGUF'), pr_revision=None, pr_num=None)